
# Customer Segmentation with RFM


1.Business Problem

2.Data Understanding

3.Data Preparation

4.Calculating RFM Metrics

5.Calculating RFM Scores

6.Creating & Analysing RFM Segments

install necessary library

In [1]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


read-get your data

In [4]:
df = pd.read_excel('/content/drive/My Drive/Github Documents/E-com_segmentation/online_retail_II.xlsx')

Quick check data tables (columns, types)

In [5]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


**Data Understanding**

In [6]:
df["Description"].nunique()

4681

In [7]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [8]:
df["Invoice"].nunique()

28816

Calculate total prices for each stock code

In [9]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df["TotalPrice"].head()

,TotalPrice
0,83.4
1,81.0
2,81.0
3,100.8
4,30.0


**Data Preparation**

For analyze eliminate "NA", 0
Change invoice data structure to find "C" (return) product

In [10]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)
df["Invoice"] = df["Invoice"].astype(str)
df = df[~df["Invoice"].str.contains("C", na=False)]

**Calculating RFM Metrics**

In [11]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [12]:
today_date = dt.datetime(2010, 12, 11)
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93


In [13]:
rfm.columns = ['recency', 'frequency', 'monetary']
rfm = rfm[rfm["monetary"] > 0]

**Calculating RFM Scores**

In [14]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

# 0-100, 0-20, 20-40, 40-60, 60-80, 80-100

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.0,91.172542,96.861457,1.00,18.0000,53.00,136.0000,374.00
frequency,4312.0,4.455705,8.170213,1.00,1.0000,2.00,5.0000,205.00
monetary,4312.0,2048.238236,8914.481280,2.95,307.9875,706.02,1723.1425,349164.35


**Creating & Analysing RFM Segments**

In [15]:
# RFM Tag
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [16]:
#Match with score
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

Now quick view segmentation for all our customers.

In [20]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0,165,11,372.86,2,5,2,25,cant_loose
12347.0,3,2,1323.32,5,2,4,52,potential_loyalists
12348.0,74,1,222.16,2,1,1,21,hibernating
12349.0,43,3,2671.14,3,3,5,33,need_attention
12351.0,11,1,300.93,5,1,2,51,new_customers


In [17]:
#Determine "new customer segment"
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index

new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

In [18]:
new_df["new_customer_id"].head()

,new_customer_id
0,12351
1,12385
2,12386
3,12427
4,12441
